# Chest X-Ray Medical Diagnosis with Deep Learning

![header image](images/xray-header-image.png)

Welcome to the first assignment of AI for Medical Diagnosis!

In this assignment you will build a state-of-the-art chest X-ray classifier using Keras for medical image diagnosis. You will:
- Pre-process and prepare a real-world X-ray dataset
- Use transfer learning to retrain a DenseNet model for X-ray image classification
- Handle class imbalance
- Compute AUROC for ROC curve
- Visualize model activity using GradCAMs

## Table of Contents
- [1. Import Packages and Functions](#1)
- [2. Load the Datasets](#2)
  - [2.1 Loading the Data](#2-1)
  - [2.2 Preventing Data Leakage](#2-2)
      - [Exercise 1 - check for Leakage](#Ex-1)
  - [2.3 Preparing Images](#2-3)
- [3. Model Development](#3)
  - [3.1 Addressing Class Imbalance](#3-1)
      - [Exercise 2 - compute Class Frequencies](#Ex-2)
      - [Exercise 3 - get Weighted Loss](#Ex-3)
  - [3.2 DenseNet121](#3-2)
- [4. Training (Optional)](#4)
  - [4.1 Training on the Larger Dataset](#4-1)
- [5. Prediction and Evaluation](#5)
  - [5.1 ROC Curve and AUROC](#5-1)
  - [5.2 Visualizing Learning with GradCAM](#5-2)


## 1. Import Packages and Functions <a name='1'></a>

We'll use:
- `numpy`, `pandas`: Data manipulation
- `matplotlib.pyplot`, `seaborn`: Visualization
- `keras` (ImageDataGenerator, DenseNet121, layers, Model)
- Custom utilities in `util.py`


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K
from keras.models import load_model

import util
from public_tests import *
from test_utils import *

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


## 2. Load the Datasets <a name='2'></a>

We'll use the ChestX-ray8 dataset (see [link](https://arxiv.org/abs/1705.02315)) and subset CSV files for train, validation, and test splits. Each image contains labels for 14 pathologies.


In [ ]:
train_df = pd.read_csv("data/nih/train-small.csv")
valid_df = pd.read_csv("data/nih/valid-small.csv")
test_df = pd.read_csv("data/nih/test.csv")
train_df.head()


In [ ]:
labels = ['Cardiomegaly', 
          'Emphysema', 
          'Effusion', 
          'Hernia', 
          'Infiltration', 
          'Mass', 
          'Nodule', 
          'Atelectasis',
          'Pneumothorax',
          'Pleural_Thickening', 
          'Pneumonia', 
          'Fibrosis', 
          'Edema', 
          'Consolidation']


### 2.2 Preventing Data Leakage <a name='2-2'></a>
Multiple images per patient: split is done at patient-level to avoid leakage. Let's check that now.


In [ ]:
def check_for_leakage(df1, df2, patient_col):
    df1_patients_unique = set(df1[patient_col].values)
    df2_patients_unique = set(df2[patient_col].values)
    patients_in_both_groups = list(df1_patients_unique.intersection(df2_patients_unique))
    leakage = len(patients_in_both_groups) != 0
    return leakage


In [ ]:
# Test for leakage between splits
print("leakage between train and valid:", check_for_leakage(train_df, valid_df, 'PatientId'))
print("leakage between train and test:", check_for_leakage(train_df, test_df, 'PatientId'))
print("leakage between valid and test:", check_for_leakage(valid_df, test_df, 'PatientId'))


### 2.3 Preparing Images <a name='2-3'></a>
Use ImageDataGenerator to create generators for training, validation, and test sets. Training uses samplewise normalization, validation/test use statistics from training sample.


In [ ]:
def get_train_generator(df, image_dir, x_col, y_cols, shuffle=True, batch_size=8, seed=1, target_w = 320, target_h = 320):
    image_generator = ImageDataGenerator(samplewise_center=True, samplewise_std_normalization=True)
    generator = image_generator.flow_from_dataframe(
        dataframe=df,
        directory=image_dir,
        x_col=x_col,
        y_col=y_cols,
        class_mode="raw",
        batch_size=batch_size,
        shuffle=shuffle,
        seed=seed,
        target_size=(target_w,target_h))
    return generator

def get_test_and_valid_generator(valid_df, test_df, train_df, image_dir, x_col, y_cols, sample_size=100, batch_size=8, seed=1, target_w = 320, target_h = 320):
    raw_train_generator = ImageDataGenerator().flow_from_dataframe(
        dataframe=train_df, 
        directory=image_dir, 
        x_col="Image", 
        y_col=labels, 
        class_mode="raw", 
        batch_size=sample_size, 
        shuffle=True, 
        target_size=(target_w, target_h))
    data_sample = raw_train_generator.next()[0]
    image_generator = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
    image_generator.fit(data_sample)
    valid_generator = image_generator.flow_from_dataframe(
        dataframe=valid_df,
        directory=image_dir,
        x_col=x_col,
        y_col=y_cols,
        class_mode="raw",
        batch_size=batch_size,
        shuffle=False,
        seed=seed,
        target_size=(target_w,target_h))
    test_generator = image_generator.flow_from_dataframe(
        dataframe=test_df,
        directory=image_dir,
        x_col=x_col,
        y_col=y_cols,
        class_mode="raw",
        batch_size=batch_size,
        shuffle=False,
        seed=seed,
        target_size=(target_w,target_h))
    return valid_generator, test_generator


In [ ]:
IMAGE_DIR = "data/nih/images-small/"
train_generator = get_train_generator(train_df, IMAGE_DIR, "Image", labels)
valid_generator, test_generator = get_test_and_valid_generator(valid_df, test_df, train_df, IMAGE_DIR, "Image", labels)


In [ ]:
# Peek at a training image
x, y = train_generator.__getitem__(0)
plt.imshow(x[0]);


## 3. Model Development <a name='3'></a>

### 3.1 Addressing Class Imbalance <a name='3-1'></a>
Let's plot positive case frequency for each label.


In [ ]:
plt.xticks(rotation=90)
plt.bar(x=labels, height=np.mean(train_generator.labels, axis=0))
plt.title("Frequency of Each Class")
plt.show()


#### Exercise 2 - Compute Class Frequencies <a name='Ex-2'></a>
Calculate positive and negative frequencies for each class.


In [ ]:
def compute_class_freqs(labels):
    N = labels.shape[0]
    positive_frequencies = np.sum(labels == 1, axis=0) / N
    negative_frequencies = np.sum(labels == 0, axis=0) / N
    return positive_frequencies, negative_frequencies


In [ ]:
freq_pos, freq_neg = compute_class_freqs(train_generator.labels)
freq_pos


In [ ]:
# Visualize positive and negative frequencies
data = pd.DataFrame({"Class": labels, "Label": "Positive", "Value": freq_pos})
data = data.append([{"Class": labels[l], "Label": "Negative", "Value": v} for l,v in enumerate(freq_neg)], ignore_index=True)
plt.xticks(rotation=90)
sns.barplot(x="Class", y="Value", hue="Label", data=data)


Use weights to balance loss:


In [ ]:
pos_weights = freq_neg
neg_weights = freq_pos
pos_contribution = freq_pos * pos_weights 
neg_contribution = freq_neg * neg_weights


In [ ]:
# Visualize weighted contributions
data = pd.DataFrame({"Class": labels, "Label": "Positive", "Value": pos_contribution})
data = data.append([{"Class": labels[l], "Label": "Negative", "Value": v} 
                        for l,v in enumerate(neg_contribution)], ignore_index=True)
plt.xticks(rotation=90)
sns.barplot(x="Class", y="Value", hue="Label", data=data);


#### Exercise 3 - Get Weighted Loss <a name='Ex-3'></a>
Return a loss function calculating weighted loss for each batch.


In [ ]:
def get_weighted_loss(pos_weights, neg_weights, epsilon=1e-7):
    def weighted_loss(y_true, y_pred):
        loss = 0.0
        for i in range(len(pos_weights)):
            loss += K.mean(-pos_weights[i]*K.log(y_pred[:,i]+epsilon)*y_true[:,i]-neg_weights[i]*K.log(1-y_pred[:,i]+epsilon)*(1-y_true[:,i]))
        return loss
    return weighted_loss


### 3.2 DenseNet121 <a name='3-2'></a>
Load pretrained DenseNet121 and add global average pooling and dense layer for multi-label sigmoid output.


In [ ]:
base_model = DenseNet121(weights='models/nih/densenet.hdf5', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(len(labels), activation="sigmoid")(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss=get_weighted_loss(pos_weights, neg_weights))


## 4. Training (Optional) <a name='4'></a>

You may train locally using model.fit_generator, but here we load pretrained weights for evaluation.


In [ ]:
model.load_weights("models/nih/pretrained_model.h5")


## 5. Prediction and Evaluation <a name='5'></a>

Get predictions for the test set.


In [ ]:
predicted_vals = model.predict_generator(test_generator, steps = len(test_generator))


### 5.1 ROC Curve and AUROC <a name='5-1'></a>

Compute ROC curve and AUROC for test results.


In [ ]:
auc_rocs = util.get_roc_curve(labels, predicted_vals, test_generator)


### 5.2 Visualizing Learning with GradCAM <a name='5-2'></a>

Visualize GradCAM heatmaps for model interpretation.


In [ ]:
df = pd.read_csv("data/nih/train-small.csv")
IMAGE_DIR = "data/nih/images-small/"
labels_to_show = np.take(labels, np.argsort(auc_rocs)[::-1])[:4]
util.compute_gradcam(model, '00008270_015.png', IMAGE_DIR, df, labels, labels_to_show)
util.compute_gradcam(model, '00011355_002.png', IMAGE_DIR, df, labels, labels_to_show)
util.compute_gradcam(model, '00029855_001.png', IMAGE_DIR, df, labels, labels_to_show)
util.compute_gradcam(model, '00005410_000.png', IMAGE_DIR, df, labels, labels_to_show)


---
## Congratulations!
You've completed the first assignment of AI for Medical Diagnosis. You learned data preprocessing, leakage detection, transfer learning, class balancing, AUROC evaluation, and model interpretability with GradCAMs.
